In [ ]:
!pip install -qU langchain langchain-google-genai langchain-community faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [34]:
import os
from google.colab import userdata

# 사용자의 API 키를 'GOOGLE_API_KEY'라는 이름의 환경변수로 설정
# 아래 코드를 실행하면 API 키를 입력할 수 있는 창이 뜹니다.
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import zipfile
import glob
import json
from google.colab import drive
from google.colab import userdata
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import time

In [36]:
def load_quiz_data_from_zips(folder_path):
    """Zip/JSON 파일에서 퀴즈 데이터를 읽어와 딕셔너리 구조로 정리합니다."""
    print(f"데이터 로드 경로: {folder_path}")
    zip_files = glob.glob(os.path.join(folder_path, "*.zip"))
    if not zip_files: return {}

    quiz_data = {}
    print(f"✅ 총 {len(zip_files)}개의 ZIP 파일을 발견했습니다.")

    for zip_file_path in zip_files:
        try:
            with zipfile.ZipFile(zip_file_path, 'r') as z:
                for filename in z.namelist():
                    if filename.endswith('.json'):
                        with z.open(filename) as f:
                            data = json.load(f)
                            for para_info in data.get('paragraphInfo', []):
                                qa_pairs = para_info.get('queAnsPairInfo', [])
                                if qa_pairs:
                                    first_qa = qa_pairs[0]
                                    age, cls, title = first_qa.get('readAge'), first_qa.get('classification'), data.get('title')
                                    if all([age, cls, title]):
                                        quiz_data.setdefault(age, {}).setdefault(cls, {}).setdefault(title, [])
                                        for qa in qa_pairs:
                                            if qa.get('question') and qa.get('ansM1'):
                                                quiz_data[age][cls][title].append({'question': qa['question'], 'answer': qa['ansM1']})
        except Exception as e:
            print(f"  -> 😭 '{os.path.basename(zip_file_path)}' 처리 중 오류: {e}")

    for age in quiz_data:
        for cls in quiz_data[age]:
            for title in quiz_data[age][cls]:
                quiz_data[age][cls][title] = [dict(t) for t in {tuple(d.items()) for d in quiz_data[age][cls][title]}]
    print("✅ 데이터 로드 및 구조화 완료!")
    return quiz_data

# --- 스크립트 시작 시 데이터 로드 실행 ---
print("프로그램에 필요한 데이터를 미리 로드합니다...")
FOLDER_PATH = "/content/drive/MyDrive/korean/korea_data2/3.개방데이터/1.데이터/Training/02.라벨링데이터"
QUIZ_DATABASE = load_quiz_data_from_zips(FOLDER_PATH)
print("-" * 50)

프로그램에 필요한 데이터를 미리 로드합니다...
데이터 로드 경로: /content/drive/MyDrive/korean/korea_data2/3.개방데이터/1.데이터/Training/02.라벨링데이터
✅ 총 15개의 ZIP 파일을 발견했습니다.
✅ 데이터 로드 및 구조화 완료!
--------------------------------------------------


In [37]:
def setup_llm_chain(prompt_template_str):
    """주어진 프롬프트 템플릿으로 LLM 체인을 설정합니다."""
    try:
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
        prompt = PromptTemplate.from_template(prompt_template_str)
        chain = prompt | llm | StrOutputParser()
        return chain
    except Exception as e:
        print(f"😭 LLM 체인 설정 중 오류 발생: {e}")
        return None

def run_toddler_quiz(book_questions, ox_chain):
    """유아 전용 O/X 퀴즈를 실행합니다."""
    lives = 4
    print("\n--- 🧠 유아용 O/X 퀴즈 ---\n문제를 잘 보고 O 또는 X로 답해줘!")
    while lives > 0:
        print(f"\n남은 기회: {'❤️' * lives}")
        base_question = random.choice(book_questions)
        original_text = f"질문: {base_question['question']} / 정답: {base_question['answer']}"
        print("AI가 퀴즈를 만들고 있어요...")
        response = ox_chain.invoke({"original_text": original_text})
        try:
            statement, correct_answer = response.strip().split('|')
        except ValueError:
            print("퀴즈 생성에 실패했어요. 다시 시도합니다."); time.sleep(2); continue
        user_answer = input(f"퀴즈: {statement}\n답변 (O/X): ").upper()
        if user_answer == correct_answer.upper():
            print("\n딩동댕! 🎉 참 잘했어요!"); return
        else:
            lives -= 1
            print(f"아쉽지만 틀렸어요. 정답은 '{correct_answer}' 였어요. ❌")
    print("\nGAME OVER... 💔 모든 기회를 사용했어요.")

def run_elementary_quiz(book_questions, ox_chain):
    """
    초등학생용 단계별 퀴즈를 실행합니다.
    [수정] 1단계와 2단계가 총 4번의 기회를 공유하도록 통합합니다.
    """
    # [수정] lives 변수를 함수 맨 위로 이동시켜 1단계와 2단계가 공유하도록 합니다.
    lives = 4

    # --- 1단계: AI O/X 퀴즈 ---
    print("\n--- 🧠 1단계: O/X 퀴즈 ---")
    print(f"현재 남은 기회: {'❤️' * lives}")
    base_question = random.choice(book_questions)
    original_text = f"질문: {base_question['question']} / 정답: {base_question['answer']}"

    print("AI가 O/X 퀴즈를 만들고 있어요...")
    response = ox_chain.invoke({"original_text": original_text})

    try:
        statement, correct_answer = response.strip().split('|')
        user_answer = input(f"퀴즈: {statement}\n답변 (O/X): ").upper()

        if user_answer == correct_answer.upper():
            print("\n딩동댕! 🎉 1단계를 통과했어요! 다음 문제로 넘어갑니다.")
        else:
            # [수정] 프로그램을 종료하는 'return' 대신, 기회를 1 감소시킵니다.
            lives -= 1
            print(f"\n아쉽지만 틀렸어요. 정답은 '{correct_answer}' 였어요. ❌")

    except ValueError:
        print("퀴즈 생성에 실패하여 1단계를 건너뜁니다.")

    # [수정] 1단계에서 기회를 모두 사용했는지 확인 후 종료
    if lives == 0:
        print("\nGAME OVER... 💔 1단계에서 모든 기회를 사용했습니다.")
        return

    # --- 2단계: 책 내용 퀴즈 (남은 기회로 진행) ---
    current_q_index = 0
    print(f"\n--- 🧠 2단계: 책 내용 퀴즈 (총 {len(book_questions)}문제) ---")

    while lives > 0 and current_q_index < len(book_questions):
        print(f"\n남은 기회: {'❤️' * lives} | 현재 {current_q_index + 1}번째 문제")

        question = book_questions[current_q_index]['question']
        answer = book_questions[current_q_index]['answer']

        user_answer = input(f"문제: {question}\n답변: ").strip()

        if user_answer == answer:
            print("정답입니다! ✅")
        else:
            lives -= 1
            print(f"틀렸습니다. ❌ 정답은 '{answer}' 입니다.")

        current_q_index += 1

    # while 루프가 끝난 후 결과를 확인합니다.
    if current_q_index == len(book_questions) and lives >= 0:
        print("\n축하합니다! 🎉 모든 문제를 풀었습니다!")
    elif lives == 0:
        print("\nGAME OVER... 💔 모든 기회를 사용했습니다. 다음에 다시 도전해주세요!")

print("✅ 퀴즈 실행에 필요한 함수들이 준비되었습니다.")

✅ 퀴즈 실행에 필요한 함수들이 준비되었습니다.


In [38]:
def main(quiz_database):
    """미리 로드된 데이터를 받아 퀴즈 프로그램을 실행합니다."""
    if not quiz_database:
        print("로드된 데이터가 없습니다. 프로그램을 실행할 수 없습니다."); return

    print("\n" + "="*50 + "\n📚 AI 동화 퀴즈 프로그램 (다시 시작) 📚")

    try:
        available_ages = list(quiz_database.keys())
        print("\n[1단계] 연령을 선택해주세요:")
        for i, age in enumerate(available_ages, 1): print(f"{i}. {age}")
        selected_age = available_ages[int(input("\n번호: ")) - 1]

        available_cls = list(quiz_database[selected_age].keys())
        print("\n[2단계] 퀴즈 분류를 선택해주세요:")
        for i, cls in enumerate(available_cls, 1): print(f"{i}. {cls}")
        selected_cls = available_cls[int(input("\n번호: ")) - 1]

        available_titles = list(quiz_database[selected_age][selected_cls].keys())
        print("\n[3단계] 책을 선택해주세요:")
        for i, title in enumerate(available_titles, 1): print(f"{i}. {title}")
        selected_title = available_titles[int(input("\n번호: ")) - 1]
    except (ValueError, IndexError):
        print("잘못된 번호입니다."); return

    selected_book_questions = quiz_database[selected_age][selected_cls][selected_title]

    if selected_age == '유아':
        easy_prompt = "당신은 유아용 동화 O/X 퀴즈 전문가입니다. 주어진 [원문]으로 O/X 퀴즈 1개를 만드세요. 퀴즈는 10~15자 이내, 쉬운 어휘를 사용하고, O/X 문제를 50% 확률로 섞어 만드세요. 거짓(X) 문장은 아이가 헷갈리지 않게 단순하게 바꾸세요. 결과는 '퀴즈 문장|정답(O 또는 X)' 형식으로만 출력하세요. 예시) 토끼는 착했다|O\n\n[원문]\n{original_text}"
        toddler_chain = setup_llm_chain(easy_prompt)
        if toddler_chain: run_toddler_quiz(selected_book_questions, toddler_chain)
    else:
        general_prompt = "주어진 [원문]의 내용으로 O/X 퀴즈 1개를 만드세요. 결과는 '퀴즈 문장|정답(O 또는 X)' 형식으로만 출력하세요. 예시) 토끼는 낮잠을 잤다|O\n\n[원문]\n{original_text}"
        elementary_chain = setup_llm_chain(general_prompt)
        if elementary_chain: run_elementary_quiz(selected_book_questions, elementary_chain)


In [42]:
# --- 프로그램 시작점 ---
if __name__ == "__main__":
    # 2번 블록에서
    미리 로드한 QUIZ_DATABASE를 사용하여 메인 함수 실행
    if 'QUIZ_DATABASE' in globals() and QUIZ_DATABASE:
        main(QUIZ_DATABASE)
    else:
        print("😭 데이터가 로드되지 않았습니다. 이전 코드 블록을 먼저 실행해주세요.")


📚 AI 동화 퀴즈 프로그램 (다시 시작) 📚

[1단계] 연령을 선택해주세요:
1. 유아
2. 초등_저학년
3. 초등_고학년

번호: 1

[2단계] 퀴즈 분류를 선택해주세요:
1. 의사소통
2. 자연탐구
3. 사회관계
4. 예술경험
5. 신체운동_건강

번호: 2

[3단계] 책을 선택해주세요:
1. 파릇파릇 초록 잎 울긋불긋 단풍잎
2. 서로 돕는 좋은 친구
3. 벌레를 잡아먹는 식물
4. 눈 속에서도 푸른 소나무
5. 단풍은 어떻게 생길까요?
6. 구름이 하늘에 그림을 그려요
7. 혀는 어떻게 맛을 느낄까? 
8. 벌나라의 일꾼벌
9. 과일들은 왜 빨갛고 노랄까요?
10. 열매나 씨가 없다면 
11. 하늘의 사냥꾼 매
12. 태양계 여덟 행성을 찾아서
13. 엉터리 빛 마술사
14. 바다는 왜 파랗고 짤까?
15. 냠냠 쩝쩝, 몸에 좋은 식물
16. 전기야, 어디만큼 왔니?
17. 바다에는 어떤 동물이 살까요?
18. 자석은 철을 좋아해!
19. 지구에 구멍이 뚫리고 있어
20. 바닷속 축구 대회
21. 굴러라, 굴러!
22. 롤러코스터가 빙글빙글 슈웅~ 
23. 떴다 떴다 비행기!
24. 컴퓨터는 어떻게 작동할까요?
25. 지구가 우리를 끌어당겨요
26. 에너지를 만드는 발전소
27. 동물들이 사라져 가요
28. 올챙이는 왜 아빠를 닮지 않았을까? 
29. 깨끗하게 싹싹싹! 
30. 배는 왜 물에 뜰까요?
31. 뱀은 어디로 갔을까? 
32. 바람은 어떻게 태어날까요?
33. 나방은 왜 불을 좋아할까? 
34. 내 알을 부탁해! 
35. 먹고 먹히는 세계
36. 땅이 우르릉 꽝! 
37. 물방울의 여행
38. 땅 속에 누가 살까? 
39. 바람 타고 둥실둥실 멀리 
40. 고무풍선이 펑! 
41. 까꿍! 날 찾아봐 
42. 불아, 물 먹으면 죽어 
43. 바람이 전하는 사막 이야기
44. 나뭇잎은 왜 초록색일까? 
45. 방아깨비 몸은 왜 초록빛일까? 
46. 너, 뭐하니? 
47. 고기는 싫어, 싫어! 
48. 킁킁! 냄새야 
49. 개는 어째서 

In [ ]:
# --- 프로그램 시작점 ---
if __name__ == "__main__":
    # 2번 블록에서 미리 로드한 QUIZ_DATABASE를 사용하여 메인 함수 실행
    if 'QUIZ_DATABASE' in globals() and QUIZ_DATABASE:
        main(QUIZ_DATABASE)
    else:
        print("😭 데이터가 로드되지 않았습니다. 이전 코드 블록을 먼저 실행해주세요.")


📚 AI 동화 퀴즈 프로그램 (다시 시작) 📚

[1단계] 연령을 선택해주세요:
1. 유아
2. 초등_저학년
3. 초등_고학년

번호: 1

[2단계] 퀴즈 분류를 선택해주세요:
1. 의사소통
2. 자연탐구
3. 사회관계
4. 예술경험
5. 신체운동_건강

번호: 1

[3단계] 책을 선택해주세요:
1. 근이와 원이의 힘겨루기
2. 뿌우가 엄마를 찾아요
3. 안녕?
4. 꼭꼭 숨어라
5. 늑대의 양을 빼앗은 사자
6. 새와 나무의 이야기
7. 말벌이 콕!
8. 좋아해 좋아해
9. 궁금하다, 궁금해!
10. 티티가 유치원에 가요
11. 은혜 갚은 호랑이
12. 키작은 딸기
13. 변덕쟁이 애벌레
14. 삐악삐악 부활절이야
15. 잘 가, 병아리야!
16. 미미의 신발이 없어졌어요
17. 키키의 친구 만들기
18. 문어의 편지
19. 예민한 오렌지
20. 와~달콤해
21. 진주를 삼킨 거위와 군자
22. 괜찮아!
23. 어부바!
24. 내 친구 퍼피를 소개해요
25. 바구니 들고 출발
26. 미안해!
27. 여러가지 모양
28. 숲 속의 세 난쟁이
29. 잠자는 숲 속의 공주
30. 멋쟁이 수박
31. 옛날에 정말 공룡이 살았을까?
32. 동글동글 지그재그 따라그려요
33. 아니야, 아니야!
34. 고미의 아침
35. 나랑 친구 할래?
36. 모양 이야기
37. 머리헌뎅이 코흘리개 눈곱쟁이
38. 요술부채
39. 누구일까요?
40. 거인의 정원
41. 열한 마리의 백조 왕자
42. 인어 공주
43. 그림에서 찾아봐
44. 단 방귀 장수
45. 독장수와 호랑이
46. 놀면서 자라요!
47. 어떤 글자가 숨어 있을까?
48. 벌거벗은 임금님
49. 새우 배는 어떤 배?
50. 소리로 계산한 고기 값
51. 뭘 해도 예쁜 우리 아기
52. 구둣방 할아버지와 요정
53. 고소한 달
54. 개구리 왕자
55. 아기 토끼가 파티에 가요
56. 콜콜, 아기 잠
57. 미운 아기 오리
58. 잭과 콩나무
59. 다섯 알의 완두콩
60. 백설 공주
61. 빨간 모자
62. 별주

In [ ]:
# --- 프로그램 시작점 ---
if __name__ == "__main__":
    # 2번 블록에서 미리 로드한 QUIZ_DATABASE를 사용하여 메인 함수 실행
    if 'QUIZ_DATABASE' in globals() and QUIZ_DATABASE:
        main(QUIZ_DATABASE)
    else:
        print("😭 데이터가 로드되지 않았습니다. 이전 코드 블록을 먼저 실행해주세요.")


📚 AI 동화 퀴즈 프로그램 (다시 시작) 📚

[1단계] 연령을 선택해주세요:
1. 유아
2. 초등_저학년
3. 초등_고학년

번호: 2

[2단계] 퀴즈 분류를 선택해주세요:
1. 의사소통
2. 자연탐구
3. 사회관계
4. 예술경험
5. 신체운동_건강

번호: 5

[3단계] 책을 선택해주세요:
1. 안전짱의 안전이 최고야!
2. 과자 사 줄게
3. 무슨 뜻 일까요?
4. 신호등이 없어요
5. 푸돌이 주사 맞는 날
6. 불장난은 안 돼요
7. 지하철을 탔어요
8. 태웅이의 사과 깍기
9. 지키면 모두 안전해요
10. 달려라, 킥보드!
11. 앗! 뜨거워요
12. 엄마, 다녀오겠습니다!
13. 예진이의 생일 파티
14. 급하다. 급해!
15. 식물원 가는 날
16. 준비 운동을 해요
17. 게으름 벌 받은 개미
18. 눈이 아파요
19. 씽씽 쌩쌩 스케이트 타기
20. 위험한 공기놀이
21. 다 함께 놀아요
22. 새콤달콤한 게 좋은걸요
23. 사탕 아저씨를 조심해요
24. 싫어요! 안 가르쳐 줄래요!
25. 아이쿠, 엉덩이야!
26. 엄마랑 약속했어요
27. 미술 시간이 좋아요
28. 엄마를 잃어버렸어요
29. 어푸어푸! 수영 놀이
30. 좋은 느낌, 싫은 느낌
31. 가스를 점검해요
32. 눈이 너무 많이 와요
33. 기침이 콜록콜록!
34. 흔들흔들 땅이 움직여요
35. 아빠랑 목욕탕에 가요
36. 내 동생이니까
37. 안전짱의 꾸르르꽝 별 구하기 대작전
38. 도와줘요! 마스콩
39. 이가 아파서 그런 거야
40. 아빠가 최고야!
41. 카라질라와 대신인형
42. 탁탁이는 못생긴 괴물이 아니야
43. 네 탓이 아니란다
44. 방귀가 자꾸 나와!
45. 할아버지랑 나랑
46. 도와줘요 119
47. 싫어요! 하지 마세요!
48. 내가 가장 잘하는 건
49. 도깨비를 이긴 아빠
50. 진짜 용기는?
51. 자꾸자꾸 배 아픈 도도
52. 누가 이상한 걸까?
53. 이상한 항아리
54. 솔방울 당번
55. 소중한 나의 집
56. 손이 큰 원숭이 차차
57.

In [ ]:
# --- 프로그램 시작점 ---
if __name__ == "__main__":
    # 2번 블록에서 미리 로드한 QUIZ_DATABASE를 사용하여 메인 함수 실행
    if 'QUIZ_DATABASE' in globals() and QUIZ_DATABASE:
        main(QUIZ_DATABASE)
    else:
        print("😭 데이터가 로드되지 않았습니다. 이전 코드 블록을 먼저 실행해주세요.")


📚 AI 동화 퀴즈 프로그램 (다시 시작) 📚

[1단계] 연령을 선택해주세요:
1. 유아
2. 초등_저학년
3. 초등_고학년

번호: 3

[2단계] 퀴즈 분류를 선택해주세요:
1. 의사소통
2. 자연탐구
3. 사회관계
4. 예술경험
5. 신체운동_건강

번호: 1

[3단계] 책을 선택해주세요:
1. 인디언과 비버
2. 베짱베짱 음악대
3. 나는야 꼬마 비행사
4. 어린왕의 마법
5. 바나나 응원단이 최고야
6. 황새의 꽁지
7. 제비의 칭찬 신문사
8. 청아의 소원
9. 난쟁이가 된 선생님
10. 눈사람 칼레
11. 미요의 노래자랑
12. 뿌뿌의 인사
13. 강아지 꿀꺽 호랑이 줄줄
14. 루루의 선택
15. 청개구리
16. 해와 달이 된 오누이

번호: 1

--- 🧠 1단계: O/X 퀴즈 ---
AI가 O/X 퀴즈를 만들고 있어요...
퀴즈: 홍수 속에서 아기 비버가 위험에 처했다
답변 (O/X): x

아쉽지만 틀렸어요. 정답은 'O' 였어요. ❌
다음에 다시 도전해주세요!
